In [ ]:
# multivariate multi-step stacked lstm example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Layer
from numpy import array
from numpy import hstack
from keras.layers import *
from keras.models import *
import seaborn as sn
from keras import backend as K
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import dataframe_image as dfi
pd.set_option("precision", 2)

from sklearn.preprocessing import PowerTransformer
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [ ]:
def format_vertical_headers(df):
    cm = sn.color_palette("blend:red,yellow,green", as_cmap=True)
    """Display a dataframe with vertical column headers"""
    styles = [dict(selector="th", props=[('width', '40px')]),
              dict(selector="th.col_heading",
                   props=[("writing-mode", "vertical-rl"),
                          ('transform', 'rotateZ(225deg)'), 
                          ('height', '90px'),
                          ('vertical-align', 'top')])]
    return (df.fillna('').style.set_table_styles(styles).background_gradient(cmap = cm,axis=None))


In [ ]:
data = pd.read_csv('..\\data\\processed\\weekly_stocks_w_fred_n_commodities.csv').set_index('Unnamed: 0')
data.index.names = ['Date']
data.dropna(inplace=True)

In [ ]:
data.columns

In [ ]:
pd.DataFrame(np.mean(data.corr(),axis=0)).sort_values(0)

In [ ]:
corr_sort = pd.DataFrame(np.mean(data.corr(),axis=0)).sort_values(0,ascending=False).index

In [ ]:
dfi.export(format_vertical_headers(pd.DataFrame(data.corr().loc[corr_sort][corr_sort])), '..\\data\\processed\\weekly_stocks_w_fred_n_commodities_corr_styled.png',max_cols=-1)

In [ ]:


format_vertical_headers(pd.DataFrame(data.corr().loc[corr_sort][corr_sort]))




In [ ]:
power = PowerTransformer(method='yeo-johnson')
 
def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape    
    
# Add attention layer to the deep learning network
class attention(Layer):
    def __init__(self, return_sequences=True):
        self.return_sequences = return_sequences
        super(attention,self).__init__()
 
    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1), 
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1), 
                               initializer='zeros', trainable=True)        
        super(attention, self).build(input_shape)
 
    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = K.tanh(K.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)   
        # Compute the weights
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = K.sum(context, axis=1)
        return context
    
def stable_sigmoid(x):
    sig = np.where(x < 0, np.exp(x)/(1 + np.exp(x)), 1/(1 + np.exp(-x)))
    return sig

def inverse_sigmoid(x):
    inv_sig = -np.log((1 / (x + 1e-8)) - 1)
    return inv_sig
    
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the dataset
		if out_end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, 0:2], sequences[end_ix-1:out_end_ix, 0][1:1+n_steps_out]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
     
# split a multivariate sequence into samples
def split_sequences_mm(sequences, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the dataset
		if out_end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)    
    
# define input sequence
#in_seq1 = np.arange(0,200,10)
#in_seq2 = np.arange(5,205,10)*100

in_seq00 = data['SPY']
in_seq01 = data['SPTM']
in_seq02 = data['SLY']
in_seq03 = data['MDYG']
in_seq04 = data['CONSUMER']
in_seq05 = data['WPUSI019011']
in_seq06 = data['WPU101']
in_seq07 = data['GVZCLS']
in_seq08 = data['UNRATE']
in_seq09 = data['VIXCLS']
in_seq10 = data['BUSLOANS']
in_seq11 = data['AWHAETP']
in_seq12 = data['UMCSENT']
in_seq13 = data['TDSP']
in_seq14 = data['DCOILWTICO']
in_seq15 = data['CPIAUCSL']
in_seq16 = data['CSUSHPINSA']
in_seq17 = data['FEDFUNDS']
in_seq18 = data['T10Y3M']
in_seq19 = data['USREC']
in_seq20 = data['USALOLITONOSTSAM']
in_seq21 = data['USPHCI']
in_seq22 = data['INDPRO']
in_seq23 = data['NEWORDER']
in_seq24 = data['PCE']
in_seq25 = data['AUDUSD=X']
in_seq26 = data['BRLUSD=X']
in_seq27 = data['BZ=F']
in_seq28 = data['CC=F']
in_seq29 = data['CL=F']
in_seq30 = data['CNYUSD=X']
in_seq31 = data['CT=F']
in_seq32 = data['EURUSD=X']
in_seq33 = data['GBPUSD=X']
in_seq34 = data['GC=F']
in_seq35 = data['GE=F']
in_seq36 = data['GF=F']
in_seq37 = data['HE=F']
in_seq38 = data['HG=F']
in_seq39 = data['HO=F']
in_seq40 = data['JPY=X']
in_seq41 = data['KC=F']
in_seq42 = data['KE=F']
in_seq43 = data['LBS=F']
in_seq44 = data['LE=F']
in_seq45 = data['NG=F']
in_seq46 = data['NQ=F']
in_seq47 = data['PA=F']
in_seq48 = data['PL=F']
in_seq49 = data['RB=F']
in_seq50 = data['SB=F']
in_seq51 = data['SI=F']
in_seq52 = data['USD']
in_seq53 = data['UUP']
in_seq54 = data['YM=F']
in_seq55 = data['ZAR=X']
in_seq56 = data['ZB=F']
in_seq57 = data['ZC=F']
in_seq58 = data['ZF=F']
in_seq59 = data['ZL=F']
in_seq60 = data['ZM=F']
in_seq61 = data['ZN=F']
in_seq62 = data['ZO=F']
in_seq63 = data['ZR=F']
in_seq64 = data['ZS=F']
in_seq65 = data['ZT=F']
in_seq66 = data['ZW=F']
in_seq67 = data['^GSPC']
in_seq68 = data['^IRX']
in_seq69 = data['^RUT']

#in_seq25 = data['VOX']
#in_seq26 = data['VCR']
#in_seq27 = data['VDC']
#in_seq28 = data['VDE']
#in_seq29 = data['VFH']
#in_seq30 = data['VHT']
#in_seq31 = data['VIS']
#in_seq32 = data['VGT']
#in_seq33 = data['VAW']
#in_seq34 = data['VNQ']
#in_seq35 = data['VPU']

in_seq = np.transpose((
    in_seq00, 
    in_seq01, 
    in_seq02, 
    in_seq03,
    in_seq04,
    in_seq05,
    in_seq06,
    in_seq07,
    in_seq08,
    in_seq09, 
    in_seq10, 
    in_seq11, 
    in_seq12,
    in_seq13,
    in_seq14,
    in_seq15,
    in_seq16,
    in_seq17,
    in_seq18, 
    in_seq19, 
    in_seq20, 
    in_seq21,
    in_seq22,
    in_seq23,
    in_seq24,
    in_seq25,
    in_seq26,
    in_seq27, 
    in_seq28, 
    in_seq29, 
    in_seq30,
    in_seq31,
    in_seq32,
    in_seq33,
    in_seq34,
    in_seq35, 
    in_seq36, 
    in_seq37, 
    in_seq38,
    in_seq39,
    in_seq40,
    in_seq41,
    in_seq42,
    in_seq43,
    in_seq44, 
    in_seq45, 
    in_seq46, 
    in_seq47,
    in_seq48,
    in_seq49,
    in_seq50,
    in_seq51,
    in_seq52,
    in_seq53, 
    in_seq54, 
    in_seq55, 
    in_seq56,
    in_seq57,
    in_seq58,
    in_seq59,
    in_seq60,
    in_seq61,
    in_seq62, 
    in_seq63, 
    in_seq64, 
    in_seq65,
    in_seq66,
    in_seq67,
    in_seq68,
    in_seq69
))

scaler = StandardScaler()

#ensuring scaling isn't applied to test data
x_train = in_seq[:-1]
scaler.fit(x_train)
in_seq = scaler.transform(in_seq)
in_seq = stable_sigmoid(in_seq)

in_seq00 = in_seq[:,0]
in_seq01 = in_seq[:,1]
in_seq02 = in_seq[:,2]
in_seq03 = in_seq[:,3]
in_seq04 = in_seq[:,4]
in_seq05 = in_seq[:,5]
in_seq06 = in_seq[:,6]
in_seq07 = in_seq[:,7]
in_seq08 = in_seq[:,8]
in_seq09 = in_seq[:,9]
in_seq10 = in_seq[:,10]
in_seq11 = in_seq[:,11]
in_seq12 = in_seq[:,12]
in_seq13 = in_seq[:,13]
in_seq14 = in_seq[:,14]
in_seq15 = in_seq[:,15]
in_seq16 = in_seq[:,16]
in_seq17 = in_seq[:,17]
in_seq18 = in_seq[:,18]
in_seq19 = in_seq[:,19]
in_seq20 = in_seq[:,20]
in_seq21 = in_seq[:,21]
in_seq22 = in_seq[:,22]
in_seq23 = in_seq[:,23]
in_seq24 = in_seq[:,24]
in_seq25 = in_seq[:,25]
in_seq26 = in_seq[:,26]
in_seq27 = in_seq[:,27]
in_seq28 = in_seq[:,28]
in_seq29 = in_seq[:,29]
in_seq30 = in_seq[:,30]
in_seq31 = in_seq[:,31]
in_seq32 = in_seq[:,32]
in_seq33 = in_seq[:,33]
in_seq34 = in_seq[:,34]
in_seq35 = in_seq[:,35]
in_seq36 = in_seq[:,36]
in_seq37 = in_seq[:,37]
in_seq38 = in_seq[:,38]
in_seq39 = in_seq[:,39]
in_seq40 = in_seq[:,40]
in_seq41 = in_seq[:,41]
in_seq42 = in_seq[:,42]
in_seq43 = in_seq[:,43]
in_seq44 = in_seq[:,44]
in_seq45 = in_seq[:,45]
in_seq46 = in_seq[:,46]
in_seq47 = in_seq[:,47]
in_seq48 = in_seq[:,48]
in_seq49 = in_seq[:,49]
in_seq50 = in_seq[:,50]
in_seq51 = in_seq[:,51]
in_seq52 = in_seq[:,52]
in_seq53 = in_seq[:,53]
in_seq54 = in_seq[:,54]
in_seq55 = in_seq[:,55]
in_seq56 = in_seq[:,56]
in_seq57 = in_seq[:,57]
in_seq58 = in_seq[:,58]
in_seq59 = in_seq[:,59]
in_seq60 = in_seq[:,60]
in_seq61 = in_seq[:,61]
in_seq62 = in_seq[:,62]
in_seq63 = in_seq[:,63]
in_seq64 = in_seq[:,64]
in_seq65 = in_seq[:,65]
in_seq66 = in_seq[:,66]
in_seq67 = in_seq[:,67]
in_seq68 = in_seq[:,68]
in_seq69 = in_seq[:,69]

out_seq = np.sum([
    in_seq00,
    in_seq01, 
    in_seq02, 
    in_seq03,
    in_seq04,
    in_seq05,
    in_seq06,    
    in_seq07,
    in_seq08,
    in_seq09,
    in_seq10, 
    in_seq11, 
    in_seq12,
    in_seq13,
    in_seq14,
    in_seq15,    
    in_seq16,
    in_seq17,
    in_seq18,
    in_seq19, 
    in_seq20, 
    in_seq21,
    in_seq22,
    in_seq23,
    in_seq24,    
    in_seq25,
    in_seq26,
    in_seq27,
    in_seq28, 
    in_seq29, 
    in_seq30,
    in_seq31,
    in_seq32,
    in_seq33,    
    in_seq34,
    in_seq35,
    in_seq36, 
    in_seq37, 
    in_seq38,
    in_seq39,
    in_seq40,
    in_seq41,    
    in_seq42,
    in_seq43,
    in_seq44,
    in_seq45, 
    in_seq46, 
    in_seq47,
    in_seq48,
    in_seq49,
    in_seq50,    
    in_seq51,
    in_seq52,
    in_seq53,
    in_seq54, 
    in_seq55, 
    in_seq56,
    in_seq57,
    in_seq58,
    in_seq59,    
    in_seq60,
    in_seq61,
    in_seq62,
    in_seq63, 
    in_seq64, 
    in_seq65,
    in_seq66,
    in_seq67,
    in_seq68,    
    in_seq69
],axis=0)
# convert to [rows, columns] structure

#in_seqs = np.array()

in_seq00 = in_seq00.reshape((len(in_seq00), 1))
in_seq01 = in_seq01.reshape((len(in_seq01), 1))
in_seq02 = in_seq02.reshape((len(in_seq02), 1))
in_seq03 = in_seq03.reshape((len(in_seq03), 1))
in_seq04 = in_seq04.reshape((len(in_seq04), 1))
in_seq05 = in_seq05.reshape((len(in_seq05), 1))
in_seq06 = in_seq06.reshape((len(in_seq06), 1))
in_seq07 = in_seq07.reshape((len(in_seq07), 1))
in_seq08 = in_seq08.reshape((len(in_seq08), 1))
in_seq09 = in_seq09.reshape((len(in_seq09), 1))
in_seq10 = in_seq10.reshape((len(in_seq10), 1))
in_seq11 = in_seq11.reshape((len(in_seq11), 1))
in_seq12 = in_seq12.reshape((len(in_seq12), 1))
in_seq13 = in_seq13.reshape((len(in_seq13), 1))
in_seq14 = in_seq14.reshape((len(in_seq14), 1))
in_seq15 = in_seq15.reshape((len(in_seq15), 1))
in_seq16 = in_seq16.reshape((len(in_seq16), 1))
in_seq17 = in_seq17.reshape((len(in_seq17), 1))
in_seq18 = in_seq18.reshape((len(in_seq18), 1))
in_seq19 = in_seq19.reshape((len(in_seq19), 1))
in_seq20 = in_seq20.reshape((len(in_seq20), 1))
in_seq21 = in_seq21.reshape((len(in_seq21), 1))
in_seq22 = in_seq22.reshape((len(in_seq22), 1))
in_seq23 = in_seq23.reshape((len(in_seq23), 1))
in_seq24 = in_seq24.reshape((len(in_seq24), 1))
in_seq25 = in_seq25.reshape((len(in_seq25), 1))
in_seq26 = in_seq26.reshape((len(in_seq26), 1))
in_seq27 = in_seq27.reshape((len(in_seq27), 1))
in_seq28 = in_seq28.reshape((len(in_seq28), 1))
in_seq29 = in_seq29.reshape((len(in_seq29), 1))
in_seq30 = in_seq30.reshape((len(in_seq30), 1))
in_seq31 = in_seq31.reshape((len(in_seq31), 1))
in_seq32 = in_seq32.reshape((len(in_seq32), 1))
in_seq33 = in_seq33.reshape((len(in_seq33), 1))
in_seq34 = in_seq34.reshape((len(in_seq34), 1))
in_seq35 = in_seq00.reshape((len(in_seq35), 1))
in_seq36 = in_seq01.reshape((len(in_seq36), 1))
in_seq37 = in_seq02.reshape((len(in_seq37), 1))
in_seq38 = in_seq03.reshape((len(in_seq38), 1))
in_seq39 = in_seq04.reshape((len(in_seq39), 1))
in_seq40 = in_seq05.reshape((len(in_seq40), 1))
in_seq41 = in_seq06.reshape((len(in_seq41), 1))
in_seq42 = in_seq07.reshape((len(in_seq42), 1))
in_seq43 = in_seq08.reshape((len(in_seq43), 1))
in_seq44 = in_seq09.reshape((len(in_seq44), 1))
in_seq45 = in_seq10.reshape((len(in_seq45), 1))
in_seq46 = in_seq11.reshape((len(in_seq46), 1))
in_seq47 = in_seq12.reshape((len(in_seq47), 1))
in_seq48 = in_seq13.reshape((len(in_seq48), 1))
in_seq49 = in_seq14.reshape((len(in_seq49), 1))
in_seq50 = in_seq15.reshape((len(in_seq50), 1))
in_seq51 = in_seq16.reshape((len(in_seq51), 1))
in_seq52 = in_seq17.reshape((len(in_seq52), 1))
in_seq53 = in_seq18.reshape((len(in_seq53), 1))
in_seq54 = in_seq19.reshape((len(in_seq54), 1))
in_seq55 = in_seq20.reshape((len(in_seq55), 1))
in_seq56 = in_seq21.reshape((len(in_seq56), 1))
in_seq57 = in_seq22.reshape((len(in_seq57), 1))
in_seq58 = in_seq23.reshape((len(in_seq58), 1))
in_seq59 = in_seq24.reshape((len(in_seq59), 1))
in_seq60 = in_seq25.reshape((len(in_seq60), 1))
in_seq61 = in_seq26.reshape((len(in_seq61), 1))
in_seq62 = in_seq27.reshape((len(in_seq62), 1))
in_seq63 = in_seq28.reshape((len(in_seq63), 1))
in_seq64 = in_seq29.reshape((len(in_seq64), 1))
in_seq65 = in_seq30.reshape((len(in_seq65), 1))
in_seq66 = in_seq31.reshape((len(in_seq66), 1))
in_seq67 = in_seq32.reshape((len(in_seq67), 1))
in_seq68 = in_seq33.reshape((len(in_seq68), 1))
in_seq69 = in_seq34.reshape((len(in_seq69), 1))

out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((
    in_seq00,
    in_seq01,
    in_seq02,
    in_seq03,
    in_seq04,
    in_seq05,
    in_seq06,
    in_seq07,
    in_seq08,    
    in_seq09,
    in_seq10,
    in_seq11,
    in_seq12,
    in_seq13,
    in_seq14,
    in_seq15,
    in_seq16,
    in_seq17,    
    in_seq18,
    in_seq19,
    in_seq20,
    in_seq21,
    in_seq22,
    in_seq23,
    in_seq24,
    in_seq25,
    in_seq26,    
    in_seq27,
    in_seq28,
    in_seq29,
    in_seq30,
    in_seq31,
    in_seq32,
    in_seq33,
    in_seq34,
    in_seq35,
    in_seq36,
    in_seq37,
    in_seq38,
    in_seq39,
    in_seq40,
    in_seq41,
    in_seq42,
    in_seq43,    
    in_seq44,
    in_seq45,
    in_seq46,
    in_seq47,
    in_seq48,
    in_seq49,
    in_seq50,
    in_seq51,
    in_seq52,    
    in_seq53,
    in_seq54,
    in_seq55,
    in_seq56,
    in_seq57,
    in_seq58,
    in_seq59,
    in_seq60,
    in_seq61,    
    in_seq62,
    in_seq63,
    in_seq64,
    in_seq65,
    in_seq66,
    in_seq67,
    in_seq68,
    in_seq69,
    out_seq))
# choose a number of time steps
n_steps_in, n_steps_out = 52*2, 13
# covert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)

#x_train = preprocessing.scale(dataset, with_mean=True, with_std=True)

print(X.shape, y.shape)
# summarize the data

"""
for i in range(len(X)):
	print(X[i], y[i])
"""

n_features = X.shape[2]
# define model
model = Sequential()
model.add(Bidirectional(LSTM(128, activation='tanh', return_sequences=True), input_shape=(n_steps_in, n_features)))
model.add((LSTM(128, activation='tanh', return_sequences=True)))
#model.add((LSTM(128, activation='tanh', return_sequences=True)))
#model.add(BatchNormalization())
model.add(attention(return_sequences=True)) #
#model.add(Dropout(0.0))
model.add(Dense(n_steps_out))

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X[:-1], y[:-1], epochs=512, verbose=0,batch_size = 128)

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

# demonstrate prediction
x_input = X[-1]
x_input = x_input.reshape((1, n_steps_in, n_features))
print(x_input)
yhat = model.predict(x_input, verbose=0, use_multiprocessing=False)

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

y_ = inverse_sigmoid(y[-1])*scaler.scale_[0]+scaler.mean_[0]
yhat = inverse_sigmoid(yhat)*scaler.scale_[0]+scaler.mean_[0]
print(y_)
print(*yhat)
plt.plot(y_)
plt.plot(*yhat)
plt.show()

rmse = mean_squared_error(y_, *yhat, squared=True)
print("RMSE:",rmse,"MAPE:",MAPE(y_, yhat))